In [ ]:
def interpolate_path(G2, mark_points):
    """查找任意两个节点间存在的不通过其他节点的路径，并做样条插值"""
    # 识别 G2 中度大于2的点作为节点
    intersections = [node for node, degree in dict(G2.degree()).items() if degree > 2]

    # 初始化路径
    interpolated_paths = []  # 插值坐标

    # 遍历所有节点对
    for i in range(len(intersections)):
        for j in range(i + 1, len(intersections)):
            node1 = intersections[i]
            node2 = intersections[j]
            
            # 移除所有node1、node2外的节点
            subintersections = [node for node in intersections if node != node1 and node != node2]
            subgraph = G2.copy()
            subgraph.remove_nodes_from(subintersections)  # 必须一次移除全部要移除的节点（边自动移除）,逐个移除会导致索引错乱
            submarks = np.delete(mark_points, subintersections)  # 一次移除所有要移除的坐标
            print("subgraph",subgraph)
            print("submarks",submarks)
            
            # 检查node1和node2之间是否存在路径
            if nx.has_path(subgraph, node1, node2):
                # 提取路径上点的索引。注意，由于子图删掉了一些点，索引会发生变化
                path = nx.shortest_path(subgraph, source=node1, target=node2)
                print("看这里",path)
                # 提取路径上点的坐标
                ordered_submarks = np.array([submarks[i] for i in path])
                
                # 提取 x, y, z 坐标
                x = ordered_submarks[:, 0]
                y = ordered_submarks[:, 1]
                z = ordered_submarks[:, 2]
                
                if len(path) > 3:
                    # 对于长度大于3的路径，进行三阶样条插值
                    t = np.linspace(0, 1, len(path))
                    spline_x = make_interp_spline(t, x, k=3)
                    spline_y = make_interp_spline(t, y, k=3)
                    spline_z = make_interp_spline(t, z, k=3)
                    # 插值点数 100
                    t_new = np.linspace(0, 1, 100)
                    spline_x_new = spline_x(t_new)
                    spline_y_new = spline_y(t_new)
                    spline_z_new = spline_z(t_new)
                    
                else:
                    # 对于长度小于等于3的路径，进行线性插值，插值点数 10
                    t_new = np.linspace(0, 1, 10)
                    spline_x_new = np.interp(t_new, [0, 1], [x[0], x[-1]])
                    spline_y_new = np.interp(t_new, [0, 1], [y[0], y[-1]])
                    spline_z_new = np.interp(t_new, [0, 1], [z[0], z[-1]])
                
                interpolated_paths.append(np.vstack([spline_x_new, spline_y_new, spline_z_new]).T)

    return interpolated_paths, intersections



    
            